<a href="https://colab.research.google.com/github/Muhammadsulton1/Pyspark_lab2/blob/main/pyspark_hw2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 39 kB/s 
     |████████████████████████████████| 199 kB 55.8 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=3f8fb409d88f11954bfa4d26919d32ce10efa11dc23135122165dc27b6721c0d
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upg

In [ ]:
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
import pyspark
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
get_ipython().system_raw('./ngrok http 4050 &')
!sleep 10
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

--2022-12-16 19:05:36--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 54.237.133.81, 54.161.241.46, 52.202.168.65, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.237.133.81|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.19M  50.7MB/s    in 0.3s    

2022-12-16 19:05:36 (50.7 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13832437/13832437]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   
https://808d-34-73-90-160.ngrok.io


In [ ]:
import numpy as np
import pandas as pd
from pyspark import SparkConf, SparkContext
from datetime import datetime
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from pyspark.sql.types import StringType
from pyspark.mllib.linalg.distributed import IndexedRow, IndexedRowMatrix
from pyspark.mllib.linalg import Vectors

#чтение данных

In [ ]:
filename_npz = '/content/drive/MyDrive/trainx16x32_0.npz'
filename_csv = '/content/trainx16x32_100.csv'

content = np.load(filename_npz)['arr_0'].astype(np.uintc)
new_content = content[:50000]
np.savetxt(filename_csv, new_content, fmt='%u', delimiter="\t")

#группируем все фильмы по юзер айди в один столбец, чтобы легче было считать tf-idf и таким образом сократим количество строк

In [ ]:
merge_watched_movie = sc.textFile('/content/trainx16x32_100.csv') \
            .map(lambda x: map(int, x.split())).groupByKey() \
            .map(lambda x : (x[0], list(x[1])))

In [ ]:
frame = merge_watched_movie.toDF(schema=["userId", "movieId"])

In [ ]:
frame.show()

+------+--------------------+
|userId|             movieId|
+------+--------------------+
|     0|[16981, 23846, 27...|
|     2|[1937, 2500, 4437...|
|     4|[27, 25419, 30135...|
|     6|[27518, 31483, 31...|
|     8|[29824, 32186, 34...|
|    10|[17393, 17827, 27...|
|    12|[2953, 3171, 1179...|
|    14|[26757, 26818, 27...|
|    16|[3171, 6389, 7407...|
|    18|[10120, 21932, 27...|
|    20|[31911, 35072, 35...|
|    22|[4332, 11791, 178...|
|    24|[175, 5184, 30434...|
|    26|[26938, 32512, 32...|
|    28|[812, 965, 978, 1...|
|    30|[31073, 31135, 33...|
|    32|[2572, 3739, 3935...|
|    34|[3902, 29491, 319...|
|    36|[11573, 44746, 50...|
|    38|[4515, 22242, 264...|
+------+--------------------+
only showing top 20 rows



#функция для генерация случайного пользователя из userId.

In [ ]:
def Generator(frame, n_uniq=1):
  uniq_elements = np.unique(frame.select('userId').distinct().collect())
  elem_number = int(np.random.choice(uniq_elements,uniq))
  return elem_number

In [ ]:
user_number = Generator(frame,uniq=1)

In [ ]:
user_number

28

#считаем tf-idf для айдишников фильмов

In [ ]:
def calc_tf_idf(frame):
  hashingTF = HashingTF(inputCol="movieId",outputCol="rawFeatures" ,numFeatures=10000)
  tf = hashingTF.transform(frame)
  tf.cache()
  idf = IDF(inputCol="rawFeatures", outputCol="features").fit(tf)
  tfidf = idf.transform(tf)
  return tfidf

In [ ]:
tfidf = calc_tf_idf(frame)

#считаем косинусное расстояние

In [ ]:
def calc_similarities(tfidf, rec_user_id=user_number):
  mat = IndexedRowMatrix(
    tfidf.select("userId", "features")\
        .rdd.map(lambda row: IndexedRow(row.userId, row.features.toArray()))).toBlockMatrix()
  cosine_matrix = mat.transpose().toIndexedRowMatrix().columnSimilarities()
  filter_cosine = cosine_matrix.entries.filter(lambda x: x.i == rec_user_id or x.j == rec_user_id)
  return filter_cosine

In [ ]:
filter_cosine = calc_similarities(tfidf)

#сортируем косинусное расстояние

In [ ]:
def sort_cosine_similarities(filter_cosine, rec_user_id=user_number):
  sort_similarity = filter_cosine.sortBy(lambda x: x.value, ascending=False) \
            .map(lambda x: IndexedRow(x.j if x.i == rec_user_id else x.i,  Vectors.dense(x.value)))
  return sort_similarity

In [ ]:
sort_similarity = sort_cosine_similarities(filter_cosine)

In [ ]:
def take_users_id(sort_similarity):
  user_id = []
  for row in sort_similarity.collect():
    user_id.append(row.index)
    return user_id

In [ ]:
user_list = take_users_id(sort_similarity)

#выдавать рекоммендации

#берем из order similiraty пользователей и обращаемся к датафрейму по юзер айди из frame и из него делаем пересечение и ищем то что не совпало и на основе этого делаем рекомендацию

In [ ]:
def get_recommend(user_list, rec_user_id=user_number, number_rec=30):
  user_id = user_list[0] #передаем из отсортировоного по убыванию первый элемент, который говорит о том
  #что файл наиболее бохожий по косинусному расстоянию
  test_user = frame.filter(frame.userId == user_id).select("movieId").rdd
  test_user = set(test_user.collect()[0].movieId)
  test_id = frame.filter(frame.userId == rec_user_id).select("movieId").rdd
  test_id = set(test_id.collect()[0].movieId)
  intersect = set.intersection(test_user,test_id)
  result = test_user - intersect
  result = list(result)
  if len(result) >= number_rec:
    return result[:number_rec]
  else:
    print("doesn't exist much films to recommendation")
    print("all movies to recommendation")
    return result[:len(result)]

In [ ]:
get_recommend(user_list)

[212994,
 188421,
 213000,
 221193,
 221195,
 229387,
 208907,
 217100,
 126989,
 225296,
 221202,
 204819,
 229394,
 229395,
 225300,
 217109,
 225302,
 380952,
 221211,
 524320,
 229410,
 204835,
 229411,
 188453,
 237605,
 217124,
 221224,
 225322,
 380971,
 204844]

In [ ]:
# a = [1,2,3,4,5,6]
# b = [1,2,5,7,8,9]
# c = set.intersection(set(a),set(b))
# print(c)
# print(set(a) - c)